In [2]:
import pyspark
from pyspark import SparkContext

sc = SparkContext()

## Exo : calcul de moyenne (slide p 37)

In [3]:
d1 = sc.parallelize([('A',42),('B',17),('A',12),('B',20)])

In [4]:
group_par_lettre = d1.groupByKey()

In [5]:
group_par_lettre.map(lambda p : (p[0],sum(p[1])/len(p[1]))).collect()

[('B', 18.5), ('A', 27.0)]

In [6]:
# 2ème solution par mapValues()
d1.groupByKey().mapValues(lambda l : sum(l)/len(l)).collect()

[('B', 18.5), ('A', 27.0)]

## Exo : soustraction d'ensembles (slide p 38)

In [7]:
d1 = sc.parallelize(['A','B','C','D'])
d2 = sc.parallelize(['A','E','C','F'])


In [8]:
d1.subtract(d2).collect()

['B', 'D']

In [9]:
# 2ème solution : moins bien car le collect peut poser problème si gros volume
# de principe le collect doit être fait à la fin
v2 = d2.collect()
d1.filter(lambda x : not x in v2).collect()

['B', 'D']

## exemple utilisation foreach - problème : pas d'affichage...

In [10]:
def p(x):
    print(x)

d1.foreach(lambda x : p(x))

## Exo : plus de followers que de followees (slide p 39)

In [11]:
graph = sc.parallelize([('A', 'B'), ('A', 'D'), ('B', 'D'), ('B', 'C'), ('C', 'E')])
revgraph = graph.map(lambda p : (p[1], p[0]))
tab = graph.cogroup(revgraph)# donne pour chacun qui il suit (followees) et qui le suit (followers)

In [12]:
def predicat(e):
    (k, v) = e
    (l1, l2) = v
    return len(l1) <= len (l2) # on veut plus de followers

In [14]:
tab.filter(predicat).map(lambda p : p[0]).collect()

['D', 'E', 'C']

In [15]:
# 2ème solution :
nb_origine = graph.map(lambda p : (p[0], 1)).reduceByKey(lambda x, y : x + y)
nb_cible = graph.map(lambda p : (p[1], 1)).reduceByKey(lambda x, y : x + y)

In [16]:
def pred2(e):
    (k, v) = e
    (v1, v2) = v
    return (v2 == None) or (v1 >= v2) #on veut plus de followers 

In [17]:
nb_cible.leftOuterJoin(nb_origine).filter(pred2).map(lambda p : p[0]).collect()

['C', 'D', 'E']

## Exo : les comptes suivis par un compte suivi par (slide p 40)

In [22]:
entree = sc.parallelize([('A',['B', 'D']), ('B', ['C', 'D']), ('C', ['E'])])

In [23]:
graph = entree.flatMap(lambda p : [(p[0], e) for e in p[1]])

In [24]:
revgraph = graph.map(lambda p : (p[1], p[0]))
# revgraph = graph.map(lambda (k, v) : (v, k)) pourquoi ça marche pas ?

In [26]:
revgraph.join(graph).map(lambda p : p[1]).groupByKey().map(lambda p : (p[0], [i for i in p[1]])).collect()

[('B', ['E']), ('A', ['C', 'D'])]

## Exo complémentaire : faire un distinct à partir de groupByKey et map

In [27]:
r = sc.parallelize(['A', 'B', 'D', 'B', 'C', 'D', 'C', 'E'])

In [28]:
r.distinct().collect()

['B', 'C', 'A', 'D', 'E']

In [29]:
r.map(lambda x : (x, 1)).groupByKey().map(lambda p : p[0]).collect()

['B', 'C', 'A', 'D', 'E']

## Exo complémentaire : faire un distinct à partir de reduceByKey et map

In [30]:
r.map(lambda x : (x, 1)).reduceByKey(lambda x, y : x).map(lambda p : p[0]).collect()

['B', 'C', 'A', 'D', 'E']

In [31]:
sc.stop()